In [1]:
import numpy as np
import pandas as pd

import random


import warnings
from sklearn.metrics import pairwise_distances


warnings.filterwarnings('ignore')

In [2]:
import scipy.sparse

In [215]:
matrix = pd.DataFrame({
    'UserId': [1,2,3,4],
    'Category_1': [0,0,2,3],
    'Category_2': [5,3,0,0],
    'Category_3': [1,0,4,7]
})
matrix = matrix.set_index('UserId')

In [216]:
matrix.head()

,Category_1,Category_2,Category_3
UserId,,,
1,0,5,1
2,0,3,0
3,2,0,4
4,3,0,7


In [217]:
y = np.array([0,1,0]).reshape(1,-1)

matrix = matrix.values
matrix

array([[0, 5, 1],
       [0, 3, 0],
       [2, 0, 4],
       [3, 0, 7]], dtype=int64)

In [218]:
y

array([[0, 1, 0]])

In [219]:
dist = pairwise_distances(matrix, y, metric='l2')    
dist = dist.flatten()
dist

array([4.12310563, 2.        , 4.58257569, 7.68114575])

In [220]:
closest = get_close_users(matrix, y, 4, 'euclidean')
closest

array([[0, 3, 0],
       [0, 5, 1],
       [2, 0, 4],
       [3, 0, 7]], dtype=int64)

In [239]:
predict(y, closest)

[2 1 0]


[2, 0]

### Distances

In [14]:
from sklearn.metrics import pairwise_distances

def get_close_users(matrix, new_user, top_n=3, metric='euclidean'):
    dist = pairwise_distances(matrix, new_user.reshape(1,-1), metric=metric)    
    dist = dist.flatten()
    idx = np.argsort(dist)[:top_n]    
    return matrix[idx]

In [15]:
def get_ids(new_user, closest, top_n=2):
    answer = []
    missing = np.where(new_user == 0)
    missing = np.array([x[1] for x in missing])
    columns_max = closest.max(axis=0)
    idx = np.argsort(columns_max)
    unsorted = np.intersect1d(missing, idx)
    buf = np.in1d(idx, missing)[::-1]
    for i,item in enumerate(buf):
        if item == True:
            answer.append(idx[::-1][i])
    
    return answer[:top_n]

In [16]:
def create_anomalies(data, delete_percentage):
    deleted_indexes = []
    anom = data.copy()
    for item in anom:
        n_delete =  delete_percentage
        positions = np.nonzero(item)
        if len(positions[0]) < 2:
            continue
        if n_delete >= len(positions[0]):
            n_delete = len(positions[0]) - 1
        indexes = random.sample(list(positions[0]), k=n_delete)
        item[indexes] = 0
        deleted_indexes.append(indexes)
    return anom, deleted_indexes

### Validation, yeeeeah

In [6]:
matrix = scipy.sparse.load_npz(r'E:\repos\Datathon\deepmenu\data\embedings.npz')

In [17]:
matrix.shape

(3001330, 9746)

In [46]:
train = matrix[:3000].toarray()
test = matrix[3000:3300].toarray()

In [47]:
corrupted, indexes = create_anomalies(test, 2)

In [48]:
from scipy.spatial import distance

In [49]:
def validate(matrix, user, original, metric):
    closest = get_close_users(matrix, user, 3, metric)
    indexes = predict(user, closest, 2)
    user[indexes] = 1
    dst = distance.euclidean(user, original)
    return dst

In [50]:
import time
distances = {}
metrics = ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan']

metrics += ['braycurtis', 'canberra', 'chebyshev', 'correlation', 'hamming','minkowski','seuclidean', 'sqeuclidean']
def check_metrics(train, test, corrupted):
    for metric in metrics:
        dist = 0
        start = time.time()
        print(metric)
        for i, cor_user in enumerate(corrupted):
            val = validate(train, cor_user, test[i], metric)
            dist+=val
        distances[metric] = dist
        print(time.time()-start)

In [ ]:
check_metrics(train, test2, corrupted)

cityblock
192.9855420589447
cosine
482.3408160209656
euclidean
212.0449936389923
l1
216.26086616516113
l2
202.44409227371216
manhattan


In [ ]:
distances

In [173]:
test = np.array([[1,2,3,4], [0,0,4,3],[5,3,0,0]])
matrix

array([[1, 2, 3, 4],
       [0, 0, 4, 3],
       [5, 3, 0, 0],
       [1, 0, 2, 0]])

In [ ]:
def get_ids(database, user, n_closest, n_items, metric):
    closest = get_close_users(database, user, n_closest, metric)
    indexes = predict(user, closest, n_items)
    return indexes

In [174]:
anomalous_data, deleted_indexs = create_anomalies(matrix, 2)